In [ ]:
import matplotlib.pyplot as plt
import numpy as np


# Plan

1. Comments on homework
2. Theory part
    1. Matrix calculus basics (3d task in test 1)
    2. Convex differentiable functions (1st task in test 1) 
    3. GD in convex setup
<!--     4. Strong convexity -->
2. kinda Real-world task: total variation denoising + in-painting
3. Comments on projects

# Comments on hw

In [ ]:
# by Nikita Akshaev
def counter(f):
    def wrapped(*args):
        wrapped.calls+=1
        return f(*args)
    wrapped.calls = 0
    return wrapped

@counter
def f(x):
    return x**6 - x**5 + x**4 - 3*x**3 + x**2 + 2.5*x - 0.1*sin(x)
    

В золотом сечении тоже можно было оптимизировать количество вызовов функции, переиспользуя прошлые вычисления

Оформляйте графики!

In [ ]:
plt.figure(figsize=(1,1))
x, y = [],[]
plt.xlabel('xlabel')
plt.ylabel('ylabel')
plt.title('title')
plt.plot(x, y, label='gold')
plt.legend()
plt.show()

# Theory part
## Matrix calculus

https://fmin.xyz/docs/theory/Matrix_calculus/

Basic approach 
![image](matrix_calculus.svg)

Since $\nabla$ is just a vector of partial derivatives, we can use all properites of derivates (derivative of sum, product, superposition, etc)

Remember vector fields from calculus ($rot$, $div$, ...)

### Just one example (Task 3 from test 1)
$$\| Ax \|^2 = \langle Ax, Ax \rangle = \langle x,A^\top Ax \rangle $$ 

(we used $\langle a, b \rangle = a^\top b$ in the last equality)

$$\nabla_x \| Ax \|^2 = \nabla_x \langle Ax, Ax \rangle = \nabla_x \langle A\overset{\downarrow}{x}, Ax \rangle +
 \nabla_x \langle Ax, A\overset{\downarrow}{x} \rangle = 2 \nabla_x \langle \overset{\downarrow}{x}, A^\top A x\rangle  = 2 A^\top A x $$

## Differentiable convex functions

$$ f(y) \geq f(x) + \langle \nabla f(x), y-x \rangle $$ 

### Other properties of differentiable convex and smooth functions
(Nesterov)
![img](convex.png)

## Gradient descent in convex setup

Nesterov 2.1.5

In L-smooth setup we obtained 

$$f(x^{k+1}) - f(x^k) \leq -w \|f'(x^k)\|^2,~~ w = h (1 - Lh/2) $$

Denote $\Delta_k = f(x^k) - f(x^*)$

Convexity : $\Delta_k \leq \langle f'(x^k), x^* - x^k \rangle \leq \|f'(x^k)\| r_0$
- We could also use it as a stopping criteria

Then $$\Delta_{k+1} \leq \Delta_k - w \frac{\Delta_k^2 }{ r_0^2} ~~~| \cdot \frac{1}{\Delta_{k+1}\Delta_{k} } $$



Telescopic inequality

...

$\Delta_0 \leq \frac{L}{2}r_0^2,~~ w = \frac{1}{2L}$

$$f(x^k) - f^* \leq \frac{2L \|x^0 - x^*\|}{k+4}$$ 

## Practice part 

In [ ]:
# from scipy import misc
# enot = (misc.face(gray = True)/255)[:, -768:]

mox = plt.imread('mox.jpg')[::3, ::3, :] / 255
image = mox

plt.imshow(image)
# plt.axis('off')
plt.show()

### Denoising problem

In [ ]:
plt.figure(figsize=(12,5))
x0 = np.clip(image +   0.6*(np.random.random(image.shape) - 0.5), 0, 1) 

plt.imshow(x0)
plt.axis('off')
plt.show()

### How to construct an optimization problem

We want to reduce the noise, while keeping resulting image close to the inital noisy picture 

General approach:  $$f(y) = \phi(y) + \|y - x \|^2_2 \rightarrow \min_y$$

How to choose the smoothing objective $\phi(x)$ ? 

### Total Variation
$l_1$-norm of difference matrix:
$$
\phi(X) = TV(X) =  \sum_{i=1}^{m-1}\sum_{j=1}^{n-1} \left\|\left[ \begin{array}{} 
X_{i+1,j} - X_{ij} \\ X_{i, j+1} - X_{ij}\end{array} \right] \right\|_2\\
$$

- the norms are not squared!
- in 1-D case its just $\sum^{n-1}_1 | x_{i+1} - x_i |$:
  if we denote 
$$D = \begin{pmatrix}
1, -1, 0, 0, \ldots,0\\
0, 1, -1, 0, \ldots, 0\\
 \ldots\\
0,0,\ldots, 0, 1, -1\\
\end{pmatrix},$$
then TV will be $TV(x) = \| Dx \|_1$, not $TV(x) = \|Dx \|_2$ !

### $l_1$-norm or $l_2$-norm 
(Boyd, Convex Optimization, 6.3.3)

<!-- ![image](l1_smooth.png) -->
<img src="l1_smooth.png" width="400"/> 

<img src="l2_smooth.png" width="400"/>

### TV is not smooth!

We will use a quick engineering hack to smooth the objective 
$$\|\cdot\|_2 \rightarrow \|\cdot\|_{2, \varepsilon} = \sqrt{\sum x_i^2 + \varepsilon}$$

More on nonsmooth optimization later

In [ ]:

smooth_eps = 1e-3
plt.figure(figsize=(14,4))
x = np.linspace(-1e-1, 1e-1, 100)
plt.subplot(121)
plt.plot(x, (x**2 + smooth_eps)**0.5, label='smoothed $l_2$ norm')
plt.plot(x, (x**2)**0.5, label='$l_2$ norm')
plt.title('$l_2$ norm smoothing')
plt.legend()

# plt.subplot(122)
# plt.plot(x, x / (x**2 + smooth_eps)**0.5)
# plt.title('derivative of smoothed norm')

### How to implement?

### Wrong approach - in-python loops

In [ ]:
%%time
x = image
m, n, k = x.shape
tv = 0
for i in range(1, m):
    for j in range(1, n):
        tv += ((x[i, j] - x[i, j-1])**2 + (x[i, j] - x[i-1, j])**2)**0.5
        


### Correct approach - using numpy array framework

In [ ]:
def TVsq_matrix(x):
    # your code

def TV(x):
    # your code

def dTV(x):
    # your code
    
    

In [ ]:
class Logger:
    def __init__(self, f):
        self.f = f
        self.calls = 0
        self.log = []
    
    def __call__(self, *args):
        self.calls += 1
        val = self.f(*args)
        self.log.append(np.linalg.norm(val))
        return val 
    
    def reset(self):
        self.calls = 0
        self.log = []

    

In [ ]:
alpha = 2

def f(x, x0):
    return TV(x) + alpha * np.linalg.norm(x - x0)
    
def df(x, x0):
    return dTV(x) + alpha * 2 * (x - x0)

lf, ldf = Logger(f), Logger(df)

In [ ]:
def gd(f, df, h, x0, niter=300, alpha=0.3, rho=0.8):
    # your code
    

In [ ]:
lf.reset(), ldf.reset()
grad = lambda x : ldf(x, x0)
fun = lambda x : lf(x, x0)
x = gd(fun, grad, 0.1, x0, niter=300)

plt.figure(figsize=(15,8))
plt.imshow(x, cmap='gray')
# print(np.linalg.norm(df(x, x0)), f(x, x0))


In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(121)
flog_ = np.array(lf.log)
flog = [flog_[0]]
for f in flog_[1:]:
    if f < flog[-1]:
        flog.append(f)
x = np.arange(1, len(flog) +1 )
yref = (flog[0] - flog[-1])/x**1.0 + flog[-1]
q = 1
def scale(y):
    return y
plt.plot(scale(flog), label='$f_k$')
plt.plot(scale(yref), label='$(f_0 - f_*)/k$')
plt.xlabel('k')
plt.ylabel('f')
plt.legend()
plt.subplot(122)
plt.plot(ldf.log, label='$||df||$')
plt.xlabel('k')
plt.ylabel('$||df||$')
plt.legend()

In [ ]:
%%time
np.random.seed(228)
# smooth_eps = 1e0
corrupted_mask = np.random.random(image.shape[:2]) < 0.98
corrupted_mask = np.expand_dims(corrupted_mask, 2)
                
x0_paint = image * (np.logical_not(corrupted_mask))# + 0.4 * np.random.random(image.shape) * corrupted_mask


def df_paint(x):
    return dTV(x) * corrupted_mask

lf_paint = Logger(TV)
ldf_paint = Logger(df_paint)

plt.figure(figsize=(16,8))
plt.subplot(121)
plt.imshow(x0_paint)

painted = gd(lf_paint, ldf_paint, 0.1, x0_paint, 3000)

plt.subplot(122)
plt.imshow(painted)

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(121)
flog_ = np.array(lf_paint.log)
flog = [flog_[0]]
for f in flog_[1:]:
    if f < flog[-1]:
        flog.append(f)
flog = np.array(flog)
# flog = np.array(lf_paint.log)#[:100]
x = np.arange(1, len(flog) +1 )
yref = 150*(flog[0] - flog[-1])/(x+4) + flog[-1]

q = 1
def scale(y):
    return (y ** (q))#[-len(y)//10:]
plt.plot(scale(flog), label='f')
# plt.plot(scale(yref)[100:], label='ref')
plt.legend()
plt.subplot(122)
plt.plot(ldf_paint.log, label='df')
plt.legend()